Q2.1.1.

Implementing Mean-Variance Optimization 

Data Collection:

a. Use historical stock price data for at least three assets (e.g., AAPL, MSFT, GOOGL)
for the past 3–5 years. You can obtain this data from sources like Yahoo Finance
(yfinance library).

In [27]:
import yfinance as yf
import pandas as pd

tickers = ["AAPL", "MSFT", "GOOGL"]
start_date = "2019-02-01"
end_date = "2024-02-01"

data = yf.download(tickers, start=start_date, end=end_date)
print(data)

[*********************100%***********************]  3 of 3 completed

Price                       Adj Close                               Close  \
Ticker                           AAPL       GOOGL        MSFT        AAPL   
Date                                                                        
2019-02-01 00:00:00+00:00   39.762127   55.728977   96.875175   41.630001   
2019-02-04 00:00:00+00:00   40.891582   56.864857   99.665108   42.812500   
2019-02-05 00:00:00+00:00   41.591206   57.385471  101.060081   43.544998   
2019-02-06 00:00:00+00:00   41.605541   55.941708   99.938446   43.560001   
2019-02-07 00:00:00+00:00   40.817543   55.095776   99.222099   42.735001   
...                               ...         ...         ...         ...   
2024-01-25 00:00:00+00:00  193.223389  151.321442  401.864929  194.169998   
2024-01-26 00:00:00+00:00  191.481918  151.640289  400.931885  192.419998   
2024-01-29 00:00:00+00:00  190.795288  152.955521  406.678894  191.729996   
2024-01-30 00:00:00+00:00  187.123276  150.912933  405.557251  188.039993   

Calculate the daily percentage returns for each stock.

In [28]:
tickers_close =  data['Adj Close']
returns = tickers_close.pct_change().dropna()
print(returns.head())

Ticker                         AAPL     GOOGL      MSFT
Date                                                   
2019-02-04 00:00:00+00:00  0.028405  0.020382  0.028799
2019-02-05 00:00:00+00:00  0.017109  0.009155  0.013997
2019-02-06 00:00:00+00:00  0.000345 -0.025159 -0.011099
2019-02-07 00:00:00+00:00 -0.018940 -0.015122 -0.007168
2019-02-08 00:00:00+00:00  0.001175 -0.003192  0.003800


b. Implement the Markowitz Mean-Variance Optimization to find the optimal
portfolio weights that maximize the Sharpe ratio


In [29]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
mu = expected_returns.mean_historical_return(tickers_close)
S = risk_models.sample_cov(tickers_close)

ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
clean_weights = ef.clean_weights()
print(clean_weights)

OrderedDict([('AAPL', 0.5813), ('GOOGL', 0.0), ('MSFT', 0.4187)])


In this snippet we use the historical mean return $\mu$ and the covariance matrix  $\sigma$ to calculate the optimal portfolio weights that maximize the sharp ratio.

Our output tells us that to maximize the Sharpe ratio as much as possible with our chosen tickers, we should put 58% of our portfolio into Apple and 42% into Microsoft, with none allocaiton going into Google.

c. Portfolio Metrics

In [30]:
expected_return, expected_volatility, sharpe_ratio = ef.portfolio_performance(risk_free_rate=0.02)

print("Optimized Portfolio Weights:")
print(clean_weights)
print("\nPortfolio Performance Metrics:")
print(f"Expected Annual Return: {expected_return:.2%}")
print(f"Expected Annual Volatility: {expected_volatility:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

Optimized Portfolio Weights:
OrderedDict([('AAPL', 0.5813), ('GOOGL', 0.0), ('MSFT', 0.4187)])

Portfolio Performance Metrics:
Expected Annual Return: 34.47%
Expected Annual Volatility: 29.27%
Sharpe Ratio: 1.18


c:\Users\gianm\AppData\Local\Programs\Python\Python310\lib\site-packages\pypfopt\efficient_frontier\efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(


In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Softmax
from tensorflow.keras.optimizers import Adam
import numpy as np

In [32]:
mean_returns = returns.mean()
cov_matrix = returns.cov()
risk_free_rate = 0.02 / 252  # Approximate daily risk-free rate

X_train = returns.mean(axis=1).values
y_train = returns.div(returns.sum(axis=1), axis=0).values 


print(len(X_train))
print(X_train)
print(y_train)
print(mu)

model = Sequential([
    Dense(16, activation='relu', input_shape=(1,)),
    Dense(16, activation='relu'),
    Dense(len(tickers), activation='softmax')  # Ensures sum of weights = 1
])

def sharpe_loss(y_true, y_pred):
    port_return = tf.reduce_sum(y_pred * mean_returns, axis=1)
    port_variance = tf.einsum('bi,ij,bj->b', y_pred, cov_matrix, y_pred)  # Batch-wise variance computation
    port_std = tf.sqrt(port_variance + 1e-6)  # Small epsilon to avoid division by zero
    sharpe_ratio = (port_return - risk_free_rate) / port_std
    return -tf.reduce_mean(sharpe_ratio) 

model.compile(optimizer=Adam(learning_rate=0.01), loss=sharpe_loss)
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Example usage
target_sharpe = 1.0  # Adjust target Sharpe ratio as needed
nn_weights = model.predict(np.array([[target_sharpe]]))
nn_weights = np.round(nn_weights,4)
print(f"Optimized Portfolio Weights for target Sharpe ratio {target_sharpe}: {nn_weights}")

1257
[ 0.02586226  0.01342038 -0.01197102 ...  0.00647387 -0.011786
 -0.04043572]
[[ 0.36611026  0.2627021   0.37118764]
 [ 0.42495631  0.22739791  0.34764577]
 [-0.0095976   0.70055412  0.30904349]
 ...
 [-0.18463319  0.44658297  0.73805023]
 [ 0.54431304  0.37768324  0.07800372]
 [ 0.15957491  0.61829256  0.22213253]]
Ticker
AAPL     0.358785
GOOGL    0.202119
MSFT     0.325215
dtype: float64
Epoch 1/20


c:\Users\gianm\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: -0.0668   
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.0698 
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.0698 
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.0698 
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.0698 
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.0698 
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.0698 
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.0698 
Epoch 9/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.0698 
Epoch 10/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.0698 
Epoch 11/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.0698 
Epoch 12/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.0698 
Epoch 13/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.0698 
Epoch 14/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.0698 
Epoch 15/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.0698

Given the nature of the portfolio wieghts that sum,we make use of a softmax layer as it is indicative to this type of design.

In [33]:
print("\nPortfolio Weigths")
nn_weights = np.array(nn_weights.flatten())
print(nn_weights)
nn_annualized_return = np.sum(nn_weights * mean_returns) * 252
nn_annualized_volatility = np.sqrt(np.dot(nn_weights.T, np.dot(cov_matrix, nn_weights)))
nn_sharpe_ratio = (nn_annualized_return - 0.02) / nn_annualized_volatility

print("\nPortfolio Performance Metrics:")
print(f"Expected Annual Return: {nn_annualized_return:.2%}")
print(f"Expected Annual Volatility: {nn_annualized_volatility:.2%}")
print(f"Sharpe Ratio: {nn_sharpe_ratio:.2f}")


Portfolio Weigths
[0.7262 0.     0.2738]

Portfolio Performance Metrics:
Expected Annual Return: 34.91%
Expected Annual Volatility: 1.88%
Sharpe Ratio: 17.50
